In [6]:
from shared_utils.clients.age_client import age_client
from shared_utils.queries.apache_age import exec_cypher

KG_NAME = "kg_83e8bf62_8797_4ed5_be17_4ae430622533"

In [ ]:
# test session creation
with age_client.scoped_session() as session:
    print("Testing exec_cypher with session")

Testing exec_cypher with session


In [8]:
# test exec cypher

with age_client.scoped_session() as session:
    cursor = exec_cypher(session,KG_NAME,"MATCH (n) RETURN n LIMIT 10")
    print("Query Result:", cursor.fetchall())

Query Result: [('{"id": 844424930131971, "label": "Entity", "properties": {"id": "AALTO UNIVERSITY", "name": "Aalto University", "tags": ["[{\\"page\\": 0, \\"document_id\\": \\"64a56fe2-2cad-4ca5-8ba9-be82feaba479\\"}]"], "references": ["2f7a035f-85fb-4c82-8a9e-68ca7c50b2c2"]}}::vertex',), ('{"id": 844424930131972, "label": "Entity", "properties": {"id": "JANE-ELLEN LONG", "name": "Jane-Ellen Long", "tags": ["[{\\"page\\": 0, \\"document_id\\": \\"64a56fe2-2cad-4ca5-8ba9-be82feaba479\\"}]"], "references": ["2f7a035f-85fb-4c82-8a9e-68ca7c50b2c2"]}}::vertex',), ('{"id": 844424930131974, "label": "Entity", "properties": {"id": "PEKKA NIKANDER", "name": "Pekka Nikander", "tags": ["[{\\"page\\": 0, \\"document_id\\": \\"64a56fe2-2cad-4ca5-8ba9-be82feaba479\\"}]"], "references": ["2f7a035f-85fb-4c82-8a9e-68ca7c50b2c2"]}}::vertex',), ('{"id": 844424930131975, "label": "Entity", "properties": {"id": "USENIX ASSOCIATION", "name": "USENIX Association", "tags": ["[{\\"page\\": 0, \\"document_id\

In [ ]:
# tested working query in psql:
# SELECT * FROM cypher('kg_83e8bf62_8797_4ed5_be17_4ae430622533', $$
# MATCH p=(a:Entity {name: 'Jane-Ellen Long'})
# RETURN a.id
# $$) as (e_id agtype);
#
# Result:
# [('"JANE-ELLEN LONG"',), ('"JANE-ELLEN LONG"',)]

cypher = "MATCH p=(a:Entity {name: %s}) RETURN a.id"
params = ("'Jane-Ellen Long'")

cypher % params

"MATCH p=(a:Entity {name: 'Jane-Ellen Long'}) RETURN a.id"

In [18]:
# test exec cypher with params
with age_client.scoped_session() as session:
    cursor = exec_cypher(conn=session, graph_name=KG_NAME, cols=None, cypher_stmt=cypher, params=params)
    print("Query Result with params:", cursor.fetchall())

Query Result with params: [('"JANE-ELLEN LONG"',), ('"JANE-ELLEN LONG"',)]


In [ ]:
entity_id = "IS A"#"844424930131971" # Entity ID of Jane-Ellen Long

# cypher / cols / params pulled from kieran's k_hops_neighbours function
cypher_k_hops = \
"MATCH p=(a:Entity {id: %s})-[r*..%s]->(b:Entity) RETURN b.name as idk, nodes(p) as idk2, r as idk3"
cols_k_hops = ("name", "nodes", "edges")
params_k_hops = (entity_id, 2)

cypher_k_hops % params_k_hops

'MATCH p=(a:Entity {id: 844424930131971})-[r*..2]->(b:Entity) RETURN b.name as idk, nodes(p) as idk2, r as idk3'

In [22]:
# test exec cypher for the k-hops query
with age_client.scoped_session() as session:
    cursor = exec_cypher(conn=session, graph_name=KG_NAME, cols=cols_k_hops, cypher_stmt=cypher_k_hops, params=params_k_hops)
    print("K-Hops Query Result:", cursor.fetchall())

K-Hops Query Result: []


In [ ]:
# Get all edges for Jane-Ellen Long 1 or 2 hops away
SELECT * FROM cypher('kg_83e8bf62_8797_4ed5_be17_4ae430622533', $$
MATCH p=(a:Entity {name: 'Jane-Ellen Long'})-[r*..2]->(b:Entity) RETURN b.name
$$) as (name agtype);

# Get all entity names
SELECT * FROM cypher('kg_83e8bf62_8797_4ed5_be17_4ae430622533', $$
MATCH (e:Entity)
RETURN e.name
$$) as (name agtype);

In [ ]:
entity_id = "'JANE-ELLEN LONG'"  # Entity ID of Jane-Ellen Long
cypher=\
"""MATCH p=(a:Entity {id: %s})-[r*..%s]->(b:Entity) RETURN b.name as idk, nodes(p) as idk2, r as idk3"""
params = (entity_id, 2)
cols = ["name", "nodes", "edges"]
cypher_stmt = cypher % params
print (cypher_stmt)
# equates to:
# SELECT * FROM cypher('kg_83e8bf62_8797_4ed5_be17_4ae430622533', $$
# MATCH p=(a:Entity {id: JANE-ELLEN LONG})-[r*..2]->(b:Entity) RETURN b.name as idk, nodes(p) as idk2, r as idk3
# $$) as (name agtype,nodes agtype,edges agtype);


# from shared_utils.queries.apache_age import build_cypher

# build_cypher("kg_83e8bf62_8797_4ed5_be17_4ae430622533", cypher_stmt, cols)

MATCH p=(a:Entity {id: JANE-ELLEN LONG})-[r*..2]->(b:Entity) RETURN b.name as idk, nodes(p) as idk2, r as idk3


In [ ]:
SELECT * FROM cypher('kg_83e8bf62_8797_4ed5_be17_4ae430622533', $$
MATCH p=(a:Entity {id: 'PROTOCOLS'})-[r*..2]->(b) RETURN b.name as idk, nodes(p) as idk2, r as idk3
$$) as (name agtype,nodes agtype,edges agtype);

In [ ]:
SELECT * FROM cypher('kg_83e8bf62_8797_4ed5_be17_4ae430622533', $$
MATCH (v)
RETURN v
$$) as (v agtype);

In [ ]:
-- Get edge count of all vertices in graph, sort descending by edge count
SELECT * FROM cypher('kg_83e8bf62_8797_4ed5_be17_4ae430622533', $$
    MATCH (n)-[r]->()
    RETURN n, COUNT(r) AS edge_count
$$) AS (n agtype, edge_count bigint)
ORDER BY edge_count DESC;